# Попытка улучшить показатели лучшей реализации с ошибкой в ~12.93% RMSE
Тоже самое, что improved_v5_feature_engineering - только чистая версия, без мусора и экспериментов.
Прошлый подход с попытками найти гиперпараметры через optuna не дал особых результатов и установленную точность так и не побили. Попробуем провести фича-инжиниринг, а именно: поиграть с тем как мы заполняем пропуски (внедрив заполнение не по mean/median, а через Predictive Imputer или KNN) и т.п.

In [1]:
from IPython.lib.deepreload import reload
from xgboost import train
%load_ext autoreload
%autoreload 2

import joblib
import numpy as np
import optuna
import pandas as pd

from catboost import CatBoostRegressor
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, root_mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, PowerTransformer, RobustScaler, QuantileTransformer, PolynomialFeatures, KBinsDiscretizer

# Вместо одного фиксированного разбиения на train/test используем стабильную стратегию кросс-валидации.
# Используем тут Cross-validation, потому что:
# 	•	нужно надёжно сравнить несколько разных моделей или гиперпараметров и понять, какая модель стабильнее и лучше в целом.
# 	•	хотим избежать случайных удач или провалов, связанных с конкретным разбиением на train/test.
# 	•	выбираем модель или гиперпараметры, которые потом будешь использовать для финального сабмишна на Kaggle.
# Делаем эту оценку, чтобы в дальнейших блокнотах-улучшениях сравнивать более корректно.
from sklearn.model_selection import KFold, cross_val_score, train_test_split

# Используем IterativeImputer:
# 	•	Он итеративно заполняет все пропуски сразу.
# 	•	Работает одновременно со всеми признаками, учитывая связи между ними.
# 	•	Не требует ручного управления порядком заполнения.
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import HistGradientBoostingRegressor

from utils.data_manager import DataManager
from utils.model_manager import ModelManager

## 1. Загрузка данных

In [2]:
data_path = 'data/home-data-for-ml-course'
train_data = pd.read_csv(data_path + '/train.csv')
test_data = pd.read_csv(data_path + '/test.csv')

## 2. Предобработка данных 

In [3]:
dm = DataManager()

### 2.1. Удаление ненужных столбцов

In [4]:
bad_columns = dm.get_all_nan_cols(train_data)
bad_columns.append('Id')
bad_columns

['Id']

In [5]:
train_data = train_data.drop(columns=bad_columns)
train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
X, y = dm.split_data_set_to_x_y(train_data, 'SalePrice')
print(X.shape, y.shape)

(1460, 79) (1460,)


### 2.2. Нормализация данных через ColumnTransformer и Pipeline
В данном кейсе мы реализуем Заполнение числовых пропусков с помощью модели (Predictive imputation). Т.е. то, что пропущено в числовых признаках - будем заполнять не медианой или средним, а будем обучать модель, которая будет предсказывать пропуски (IterativeImputer + RandomForestRegressor)

In [7]:
RANDOM_STATE = 42

In [8]:
# Получение числовых колонок
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns

# Получение нечисловых колонок (всех остальных)
non_numeric_columns = X.select_dtypes(exclude=['float64', 'int64']).columns

numeric_columns, non_numeric_columns

(Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
        'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
        'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
        'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
        'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
        'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
        'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
        'MoSold', 'YrSold'],
       dtype='object'),
 Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
        'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
        'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
        'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
        'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
        'Heating', 'Hea

In [9]:
# Создаем preprocessor с разными трансформерами для разных типов данных
# Числовые данные пропущенные предсказываем с помощью модели RandomForestRegressor

# Пайплайн для числовых признаков (итеративное заполнение)

numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(
        estimator=RandomForestRegressor(n_estimators=50, random_state=RANDOM_STATE),
        max_iter=10,
        random_state=RANDOM_STATE
    ))
])

# Пайплайн для категориальных признаков (заполнение частым значением и кодирование)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Объединяем препроцессоры
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, non_numeric_columns)
    ],
    remainder='drop'
)

default_params = {
    'iterations': 1000, 
    'learning_rate': 0.05, 
    'depth': 6, 
    'loss_function': 'RMSE', 
    'verbose': 0, 
    'random_seed': RANDOM_STATE,
}

### 2.3. Разбиение данных на обучающую и тестовую выборки

In [10]:
# Катбуст сам выбирает даныне для валидации, поэтому отсекаем данные только для теста
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

# Проверка размера полученных наборов
print(f'Train size: {len(X_train)}')
print(f'Test size: {len(X_test)}')

Train size: 1022
Test size: 438


## Пайплайн делаем по уму, чтобы не ждать кучу часов SFS, разносим все по шагам и переиспользуем
НЕ ЗАПУСКАЬ ячейку ниже, если только не надо переделать результаты подбора фич. Сделано и сохранено в PKL в прошлом блокноте. Работает 20+ часов(!)

In [ ]:
# Обучаем препроцессор отдельно
preprocessor.fit(X_train)

# Преобразуем данные
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Получаем имена признаков после препроцессинга
feature_names = preprocessor.get_feature_names_out()

# Обучаем SequentialFeatureSelector один раз
sfs = SequentialFeatureSelector(
    estimator=CatBoostRegressor(**default_params),
    n_features_to_select='auto',
    direction='backward',
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1
)
sfs.fit(X_train_processed, y_train)

# Сохраняем препроцессор и отбор фич
joblib.dump(preprocessor, 'preprocessor.pkl')
joblib.dump(sfs, 'feature_selector.pkl')

## 3. Загруаем препроцессор и выбранные фичи с помощью SequentialFeatureSelector с прошлого блокнота (improved_v5_feature_engineering)

In [11]:
# Проверяем, какие фичи были отобраны
# Загружаем сохранённые шаги
preprocessor = joblib.load('preprocessor.pkl')
sfs = joblib.load('feature_selector.pkl')
# Получаем маску выбранных фич
support_mask = sfs.get_support()

# Получаем имена фич после препроцессора
if hasattr(preprocessor, 'get_feature_names_out'):
    feature_names = preprocessor.get_feature_names_out()
else:
    feature_names = [f"feature_{i}" for i in range(X_processed.shape[1])]

# Выбранные признаки:
selected_features = feature_names[support_mask]
selected_features, len(selected_features), len(feature_names)

(array(['num__OverallQual', 'num__OverallCond', 'num__YearBuilt',
        'num__YearRemodAdd', 'num__1stFlrSF', 'num__LowQualFinSF',
        'num__GrLivArea', 'num__BsmtFullBath', 'num__FullBath',
        'num__HalfBath', 'num__KitchenAbvGr', 'num__Fireplaces',
        'num__GarageCars', 'num__EnclosedPorch', 'num__3SsnPorch',
        'num__ScreenPorch', 'num__PoolArea', 'cat__MSZoning_C (all)',
        'cat__MSZoning_FV', 'cat__Street_Pave', 'cat__Alley_Grvl',
        'cat__LotShape_IR2', 'cat__LotShape_IR3', 'cat__LandContour_Bnk',
        'cat__LandContour_HLS', 'cat__LandContour_Low',
        'cat__LandContour_Lvl', 'cat__LotConfig_FR2', 'cat__LandSlope_Sev',
        'cat__Neighborhood_BrDale', 'cat__Neighborhood_BrkSide',
        'cat__Neighborhood_Crawfor', 'cat__Neighborhood_Edwards',
        'cat__Neighborhood_Gilbert', 'cat__Neighborhood_IDOTRR',
        'cat__Neighborhood_MeadowV', 'cat__Neighborhood_Mitchel',
        'cat__Neighborhood_NPkVill', 'cat__Neighborhood_NWAmes',
 

In [12]:
# Загружаем и собираем финальный пайплайн для быстрого обучения
# Кастомный трансформер для применения сохранённого препроцессора и SFS
class PreprocessAndSelect(BaseEstimator, TransformerMixin):
    def __init__(self, preprocessor, selector):
        self.preprocessor = preprocessor
        self.selector = selector
        
    def fit(self, X, y=None):
        return self  # уже обучены
        
    def transform(self, X):
        X_transformed = self.preprocessor.transform(X)
        X_selected = self.selector.transform(X_transformed)
        return X_selected

# Загружаем сохранённые шаги
preprocessor = joblib.load('preprocessor.pkl')
sfs = joblib.load('feature_selector.pkl')

# Собираем финальный пайплайн
final_fast_pipeline = Pipeline([
    ('preprocess_and_select', PreprocessAndSelect(preprocessor, sfs)),
    ('model', CatBoostRegressor(**default_params))
])

# Обучение финальной модели (быстро!)
final_fast_pipeline.fit(X_train, y_train)

# Предсказание
y_pred = final_fast_pipeline.predict(X_test)

# Метрики
print("RMSE:", root_mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))

RMSE: 24789.95229804395
R²: 0.9119327203112025


#### К чему стремимся:
=== CatBoost ===
- RMSE: 23465.1123
- R² (коэффициент детерминации): 0.9211
- Средний RMSE на кросс-валидации: 25687.30318717033

In [13]:
# Сохраняем всю цепочку: препроцессинг + отбор фич + модель
joblib.dump(final_fast_pipeline, 'catboost_pipeline_with_sfs_clean.pkl')

['catboost_pipeline_with_sfs_clean.pkl']

## 4. Формируем Submission файл для Kaggle обучив модель на 100% данных

In [14]:
final_fast_pipeline.fit(X, y)

Pipeline(steps=[('preprocess_and_select',
                 PreprocessAndSelect(preprocessor=ColumnTransformer(transformers=[('num',
                                                                                   Pipeline(steps=[('imputer',
                                                                                                    IterativeImputer(estimator=RandomForestRegressor(n_estimators=50,
                                                                                                                                                     random_state=42),
                                                                                                                     random_state=42))]),
                                                                                   Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrAr...
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object'))]),
                                     selector=SequentialFeatureSelector(direction='backward',
                                                                        estimator=<catboost.core.CatBoostRegressor object at 0x16a858170>,
                                                                        n_jobs=-1,
                                                                        scoring='neg_root_mean_squared_error'))),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x16a66f920>)])

In [15]:
X_submission = test_data.drop(columns=bad_columns)
X_submission.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [16]:
y_submission = final_fast_pipeline.predict(X_submission)
y_submission

array([119807.79064939, 144281.59764853, 180486.81736903, ...,
       160087.12734036, 110371.06086964, 218582.58681826])

In [17]:
submission = pd.DataFrame({
    'Id': test_data['Id'],
    'SalePrice': y_submission
})
submission.set_index('Id', inplace=True)
submission.head()

,SalePrice
Id,
1461,119807.790649
1462,144281.597649
1463,180486.817369
1464,197361.753544
1465,181643.097312


In [18]:
submission.to_csv('submission_feature_engineered_v5_cleaned.csv')

# Выводы
Показатели улучшились! При внедрении IterativeImputera, который использует RandomForestRegression для обучения модели для заполнения пропущенных числовых фич получили прирост к точности.

Внедрение подбора фич с помощью SequentialFeatureSelector не показало себя НИКАК (!). Процесс чудовищно долгий (20+ часов) - результаты - нулевые. Показатели ухудшились (!).